In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Insert the directory
import sys
sys.path.insert(0,"/content/drive/My Drive/TSOAI/S6")

In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchsummary import summary
#from model import Net
from model import train,test,Net5
from utils import display_model_summary
from matplotlib import pyplot as plt

In [8]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [ ]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [10]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
# instantiate the model and load it to the device
model = Net5().to(device)
#model
display_model_summary(model)

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
            Conv2d-4           [-1, 12, 24, 24]           1,080
              ReLU-5           [-1, 12, 24, 24]               0
       BatchNorm2d-6           [-1, 12, 24, 24]              24
            Conv2d-7           [-1, 12, 22, 22]           1,296
              ReLU-8           [-1, 12, 22, 22]               0
           Dropout-9           [-1, 12, 22, 22]               0
      BatchNorm2d-10           [-1, 12, 22, 22]              24
        MaxPool2d-11           [-1, 12, 11, 11]               0
           Conv2d-12           [-1, 12, 11, 11]             144
             ReLU-13           [-1, 12, 11, 11]               0
          Dropout-14           [-1

In [ ]:
# set up the optimizer with model parameters , learning rate and momentum
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

In [ ]:
# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []
# Number of Epochs or how many times the entire data is used for training
num_epochs = 15
for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  ## Send the model , device , train_loader,optimizer and loss function to the train function
  train_accuracy,train_loss = train(model, device, train_loader, optimizer,epoch)
  # Append the train accuracy and test loss value of the ith epoch
  train_acc.append(train_accuracy)
  train_losses.append(train_loss)

  ## Send the model,device,test_loader and loss function
  test_accuracy,test_loss = test(model, device, test_loader)
  #store the test and loss and test accuracy for this epoch
  test_losses.append(test_loss)
  test_acc.append(test_accuracy)
  #  learning rate of the optimizer will be adjusted according to the predefined schedule
  scheduler.step()

Epoch 1


Loss=0.011988081969320774 Accuracy=99.36: 100%|██████████| 938/938 [00:26<00:00, 35.99it/s]



Test set: Average loss: 0.0201, Accuracy: 9936/10000 (99.36%)

Epoch 2


Loss=0.009608530439436436 Accuracy=99.42: 100%|██████████| 938/938 [00:25<00:00, 36.57it/s]



Test set: Average loss: 0.0199, Accuracy: 9936/10000 (99.36%)

Epoch 3


...


Test set: Average loss: 0.0193, Accuracy: 9940/10000 (99.40%)

Epoch 14


Loss=0.00021813108469359577 Accuracy=99.47: 100%|██████████| 938/938 [00:25<00:00, 36.36it/s]



Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99.39%)

Epoch 15


Loss=0.10083936899900436 Accuracy=99.50: 100%|██████████| 938/938 [00:26<00:00, 35.62it/s]



Test set: Average loss: 0.0192, Accuracy: 9940/10000 (99.40%)



Target

1.	Improved Accuracy of the testing
1.	Let’s change the LR using scheduler
2.	After 6th epoch we are reducing by factor of gamma

Number of Parameters	6422

Best Training Accuracy	99.50

Best Test Accuracy	99.40

Number of Epoch	15

Model Name 	Net5

Analysis :
The network is constantly hitting 99.40 with some amount of over-fitting..

Submitting the model as it's meeting the objective

